# Data Preparation with SageMaker Data Wrangler (Part 4)
> A detailed guide on AWS SageMaker Data Wrangler to prepare data for machine learning models. This is a five parts series where we will prepare, import, explore, process, and export data using AWS Data Wrangler. You are reading **Part 4:Preprocess data using Data Wrangler**.

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker, wrangler]
- image: images/copied_from_nb/images/2022-05-25-aws-sagemaker-wrangler-p4.jpeg

![](images/2022-05-25-aws-sagemaker-wrangler-p4.jpeg)

# Enviornment

This notebook is prepared with Amazon SageMaker Studio using `Python 3 (Data Science)` Kernel and `ml.t3.medium` instance.

# About
This is a detailed guide on using **AWS SageMaker Data Wrangler** service to prepare data for machine learning models. SageMaker Data Wrangler is a multipurpose tool with which you can
* import data from multiple sources
* explore data with visualizations
* apply transformations
* export data for ml training

This guide is divided into five parts
* [Part 1: Prepare synthetic data and place it on multiple sources](https://hassaanbinaslam.github.io/myblog/aws/ml/sagemaker/2022/05/17/aws-sagemaker-wrangler-p1.html)
* [Part 2: Import data from multiple sources using Data Wrangler](https://hassaanbinaslam.github.io/myblog/aws/ml/sagemaker/2022/05/23/aws-sagemaker-wrangler-p2.html)
* [Part 3: Explore data with Data Wrangler visualizations](https://hassaanbinaslam.github.io/myblog/aws/ml/sagemaker/2022/05/24/aws-sagemaker-wrangler-p3.html)
* **Part 4: Preprocess data using Data Wrangler (You are here)**
* Part 5: Export data for ML training

# Part 4: Preprocess data using Data Wrangler

We will continue from where we left in [part-3](https://hassaanbinaslam.github.io/myblog/aws/ml/sagemaker/2022/05/24/aws-sagemaker-wrangler-p3.html). Open [customer-churn.flow](https://github.com/hassaanbinaslam/myblog/blob/master/_notebooks/datasets/2022-05-23-aws-sagemaker-wrangler-p2/customer-churn-p3.flow) file in AWS SageMaker Data Wrangler console. Once opened our flow will look like this

![]()